In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import os

# Imports from local scripts.
from data_downloader import download, get_filename
from trailing_stop_loss import generate_stop_loss_levels
from trailing_stop_loss import calculate_pl

In [ ]:
ticker = 'AAPL'
interval = '5s'
year = '2024'

In [ ]:
# get data
filename = get_filename(ticker, interval, year)
# If file doesn't exist, then download it.
if not os.path.exists(filename):
	download(ticker, interval, year)

df = pd.read_parquet(filename)

# # show what is loaded
# fig = px.line(df[-1000:], y='open', title=f'{ticker} Open Prices')
# fig.show()

total_bars = df.shape[0]
print(f'Total bars: {total_bars} ({total_bars:,})')

In [ ]:
# Set trailing stop-loss levels
max_sl = 0.3
sl_steps = 10
stop_loss_percents, stop_loss_levels = generate_stop_loss_levels(max_sl, sl_steps)

print("Stop-loss levels (in percent):")
print(stop_loss_percents)
print("\nStop-loss levels (as fractions):")
print(stop_loss_levels)

In [ ]:
# New cell - Calculate results using calculate_pl
tsl_profits = calculate_pl(df, stop_loss_levels)
print(f"Results shape: {tsl_profits.shape}")
print(f"Shape interpretation: ({len(stop_loss_levels)} SL levels, {len(df)} entry points)")

In [ ]:
from pattern_searcher import PatternSearcher
from pattern_analysis import create_window

start_index = 1192098
m = 60 # window size
r_limit = 0.98
searcher = PatternSearcher(df['open'], template_length=m)
pattern = create_window(df, start_index, m)
r = searcher.search(pattern) # correlations
mask = r > r_limit
found_similar_count = len(r[mask])
indices = np.where(mask)[0]
indicesString = np.array2string(indices, threshold=10)
print(f'{len(indices)} points with r above {r_limit}: {indicesString}')

In [ ]:
# Create trades.csv from similar pattern points
trades_data = []

# Use 0.15% trailing stop loss (position 5 in stop_loss_levels)
tsl_index = 5  # 0.15% stop loss level
print(f"Using TSL level: {stop_loss_percents[tsl_index]:.2f}% ({stop_loss_levels[tsl_index]:.4f})")

for i, pattern_start_idx in enumerate(indices, 1):
    # Entry point is after the pattern window (pattern_start + window_size)
    entry_idx = pattern_start_idx + m
    
    # Skip if entry point is beyond available data
    if entry_idx >= len(df):
        continue
    
    # Get entry data
    timestamp = df.iloc[entry_idx]['timestamp']
    entry_price = df.iloc[entry_idx]['open']
    
    # Get PnL from tsl_profits for this entry point and TSL level
    pnl = tsl_profits[tsl_index, entry_idx] if entry_idx < tsl_profits.shape[1] else np.nan
    
    trades_data.append({
        'trade_id': i,
        'side': 'long',
        'entry_ts': pd.to_datetime(timestamp, unit='s'),
        'entry_price': entry_price,
        'exit_ts': pd.NaT,  # Will be filled later if needed
        'exit_price': np.nan,
        'stop_loss': stop_loss_percents[tsl_index] / 100,  # Convert to fraction
        'take_profit': 0.25,
        'pnl': pnl,
        'pnl_pct': (pnl / entry_price) * 100 if not np.isnan(pnl) else np.nan,
        'mae': np.nan,
        'mfe': np.nan,
        'duration_s': np.nan,
        'params_hash': f'PATTERN_{r_limit}'
    })

# Create DataFrame and save to CSV
trades_df = pd.DataFrame(trades_data)
trades_df.to_csv('data/trades.csv', index=False)

print(f"Created trades.csv with {len(trades_data)} pattern-based entries")
print(f"Average PnL: {trades_df['pnl'].mean():.4f}")
print(f"Win rate: {(trades_df['pnl'] > 0).mean():.2%}")
print("\nFirst 3 rows:")
print(trades_df.head(3))